# Steady State Behavior in FMC Models

A key assumption of the timelag ODE is that the state of the system will approach the equilibrium moisture content if environmental conditions are kept stable. Many physics-based models of dynamic systems have this type longrun behavior. 

In this notebook, we will investigate how a trained RNN behaves when predicting new values with constant weather inputs. It is possible that the network learned some type of trend or oscillatory behavior that is built into the weights. If, however, the hidden states stabilize to some constant level associated with a constant output, those hidden states could be extracted and used for initialization.

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sys
sys.path.append("../src")
from models.moisture_rnn import RNNData
import pandas as pd
from utils import str2time
from data_funcs import cv_data_wrap

In [ ]:
# Read in trained model
rnn = tf.keras.models.load_model("../models/train_rocky/rnn.keras")

features_list = ['Ed', 'Ew', 'solar', 'wind', 'elev', 'lon', 'lat', 'rain', 'hod', 'doy'] 

In [ ]:
tstart = str2time("2023-01-01T00:00:00Z")
tend = str2time("2023-12-31T23:00:00Z")

dat = pd.read_pickle("../models/train_rocky/ml_data.pkl")

In [ ]:
rnn.summary()

## Training Data Summary

Get sample means and bounds of data used to train model before scaling.

In [ ]:
train, val, test = cv_data_wrap(dat, fstart=None, fend=None, tstart=tstart, tend=tend, val_hours=48, test_frac = 0.1, random_state=42)
rnndat = RNNData(train, val, test=None, method="random", timesteps=48, random_state=None, features_list = features_list)

In [ ]:
arr = rnndat.X_train
df = pd.DataFrame(arr.reshape(-1, arr.shape[2]), columns=features_list)
train_stats = df.agg(['mean', 'min', 'max', 'std']).T  
train_stats.index.name = 'Variable'
train_stats = pd.concat(
    [pd.DataFrame([np.mean(rnndat.y_train)], index=['fm'], columns=['mean'])
       .assign(min=np.min(rnndat.y_train), max=np.max(rnndat.y_train), std=np.std(rnndat.y_train)),
     train_stats]
)
train_stats

In [ ]:
rnndat.scale_data()

## Test Longrun Behavior

Based on features list, set up constant inputs. Since standard scaling used, start with zeros input to represent mean of all features.

In [ ]:
tsteps = 500
X0 = np.zeros((1, tsteps, len(features_list)))

In [ ]:
p0 = rnn.predict(X0)

In [ ]:
plt.plot(np.arange(0, tsteps), p0[0,:,0])
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Long-run behavior with constant zeros")

In [ ]:
print(f"Last 10 values: {p0[0,-10:,0]}")

In [ ]:
plt.plot(np.arange(0, 30), p0[0,0:30,0])
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Starting timesteps")

## Test HOD

Hour of day (HOD) was used as a feature to help the model learn the diurnal patterns of FMC. Next, we test the long run behavior of the RNN predictions when all features are kept to zero (mean of training data) and HOD cycles from 0-23 (scaled to match training)

In [ ]:
tsteps = 500
seq = np.arange(24) # 0-23
repeats = int(np.ceil(tsteps / len(seq)))
hod = np.tile(seq, repeats)[:tsteps]

# Scale hod column with rnndata scaler, but only copy that column over to the zeros
from models.moisture_rnn import scale_3d
ind = features_list.index("hod")
X0 = np.zeros((1, tsteps, len(features_list)))
Xtemp = X0.copy()
Xtemp[:,:,ind] = hod
Xtemp = scale_3d(Xtemp, rnndat.scaler, fit=False)
Xh = X0.copy()
Xh[:,:,ind] = Xtemp[:,:,ind]

In [ ]:
ph = rnn.predict(Xh)

In [ ]:
plt.plot(np.arange(0, tsteps), ph[0,:,0])
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Constant zeros, cycling HOD")

In [ ]:
print(Xh[:,456:(456+24), ind])

In [ ]:
t = 456
plt.plot(np.arange(0, tsteps)[t:t+48], ph[0,t:t+48,0])
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Constant zeros, cycling HOD")

## Test DOY

The day of the year is used as a feature to help the model learn seasonal patterns in FMC. Next, we test the long run behavior of the RNN when all features are kept to zero, but the day of the year cycles from 1-365.

In [ ]:
tsteps = 365*5
seq = np.arange(365) # 0-364
repeats = int(np.ceil(tsteps / len(seq)))
doy = np.tile(seq, repeats)[:tsteps]

# Scale hod column with rnndata scaler, but only copy that column over to the zeros
from models.moisture_rnn import scale_3d
ind = features_list.index("doy")
X0 = np.zeros((1, tsteps, len(features_list)))
Xtemp = X0.copy()
Xtemp[:,:,ind] = doy
Xtemp = scale_3d(Xtemp, rnndat.scaler, fit=False)
Xd = X0.copy()
Xd[:,:,ind] = Xtemp[:,:,ind]

In [ ]:
pd = rnn.predict(Xd)

In [ ]:
plt.plot(np.arange(0, tsteps), pd[0,:,0])
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Constant zeros, cycling DOY")

In [ ]:
plt.plot(np.arange(0, tsteps)[-365:], pd[0,-365:,0])
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Constant zeros, cycling DOY")

## Test Eqs

Test varying Ed and Ew, and whether they interact at all.

In [ ]:
ind = features_list.index("Ed")
ind2 = features_list.index("Ew")

In [ ]:
sdgrid = [-3, -2, -1, -0.5, 0, 0.5, 1, 2, 3]
X_list = []
for val in sdgrid:
    arr = np.zeros((1, tsteps, len(features_list)))
    arr[:, :, ind] += val
    arr[:, :, ind2] += val
    X_list.append(arr)

X = np.concatenate(X_list, axis=0)

In [ ]:
p = rnn.predict(X)

In [ ]:
# Get blue-red color spectrum
import matplotlib.colors as mcolors
norm = mcolors.Normalize(vmin=min(sdgrid), vmax=max(sdgrid))
cmap = plt.colormaps.get_cmap('coolwarm')  # blue → red

plt.figure(figsize=(10,6))
for i in range(0, X.shape[0]):
    color = cmap(norm(sdgrid[i]))
    line, = plt.plot(np.arange(0, tsteps), p[i, :, 0], color=color)
    plt.text(
        tsteps - 1,
        p[i, -1, 0],    
        f"Eq={sdgrid[i]}",
        va='center', ha='left',
        color=color
    ) 
plt.xlim(0, tsteps*1.1)
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Varying Constant Equilibria")

## Test Elev

Run to stead state with all zeros except different levels of elevation

In [ ]:
tsteps=500
ind = features_list.index("elev")
sdgrid = [-3, -2, -1, -0.5, 0, 0.5, 1, 2, 3]
X_list = []
for val in sdgrid:
    arr = np.zeros((1, tsteps, len(features_list)))
    arr[:, :, ind] += val
    X_list.append(arr)

X = np.concatenate(X_list, axis=0)

In [ ]:
p = rnn.predict(X)

In [ ]:
norm = mcolors.Normalize(vmin=min(sdgrid), vmax=max(sdgrid))
cmap = plt.colormaps.get_cmap('coolwarm')  # blue → red

plt.figure(figsize=(10,6))
for i in range(0, X.shape[0]):
    color = cmap(norm(sdgrid[i]))
    line, = plt.plot(np.arange(0, tsteps), p[i, :, 0], color=color)
    plt.text(
        tsteps - 1,
        p[i, -1, 0],    
        f"Elev={sdgrid[i]}",
        va='center', ha='left',
        color=color
    ) 
plt.xlim(0, tsteps*1.1)
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Varying Elevation")

## Test Rain

Holding all else to zeros, vary rain. 

What do we expect to happen? Some scenarios:
- constant rain of various levels, expect moisture to increase to saturation level
- impulse of rain, need to allow stabilization before

In [ ]:
tsteps=500
ind = features_list.index("rain")
sdgrid = [-3, -2, -1, -0.5, 0, 0.5, 1, 2, 3, 10, 50]
X_list = []
for val in sdgrid:
    arr = np.zeros((1, tsteps, len(features_list)))
    arr[:, :, ind] += val
    X_list.append(arr)

X = np.concatenate(X_list, axis=0)

In [ ]:
p = rnn.predict(X)

In [ ]:
norm = mcolors.Normalize(vmin=min(sdgrid), vmax=max(sdgrid))
cmap = plt.colormaps.get_cmap('coolwarm')  # blue → red

plt.figure(figsize=(10,6))
for i in range(0, X.shape[0]):
    color = cmap(norm(sdgrid[i]))
    line, = plt.plot(np.arange(0, tsteps), p[i, :, 0], color=color)
    plt.text(
        tsteps - 1,
        p[i, -1, 0],    
        f"Rain={sdgrid[i]}",
        va='center', ha='left',
        color=color
    ) 
plt.xlim(0, tsteps*1.1)
plt.xlabel("Time Step")
plt.ylabel("FMC Prediction (%)")
plt.title("Varying Rain")